In [1]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv(r'C:\Users\acer pc\OneDrive\Desktop\Placement_Data_Full_Class.csv')

In [5]:
data.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [8]:
dataset = data.copy()

In [9]:
dataset.drop(['salary'],axis=1, inplace=True)

In [13]:
for i in range(len(dataset)):
    if(dataset.loc[i, 'status'] == 'Placed'):
        dataset.loc[i, 'status'] = 1
    else: dataset.loc[i, 'status'] = 0
        
dataset['status'] = dataset['status'].astype('int')

In [14]:
X = dataset.drop(['status'], axis=1)
y = dataset['status']


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [18]:
numerical_col = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p']
categorical_col = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation']

In [19]:

#inpute numerical missing data with median
numerical_transformer = make_pipeline(SimpleImputer(strategy='median'),
                                      StandardScaler())

#inpute categorical data with the most frequent value of the feature and make one hot encoding
categorical_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                        OneHotEncoder(handle_unknown='ignore'))

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_col),
                                               ('cat', categorical_transformer, categorical_col)])


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [21]:
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingClassifier())])

In [22]:
param_grid = {'model__learning_rate':[0.001, 0.01, 0.1], 
              'model__n_estimators':[100, 150, 200, 300, 350, 400]}

#param_grid = {'model__learning_rate':[0.1], 
#              'model__n_estimators':[150]}

#use recall score
grid = GridSearchCV(clf, param_grid, cv=10, scoring='accuracy', n_jobs=-1)

In [23]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                             

In [24]:
grid.best_params_

{'model__learning_rate': 0.01, 'model__n_estimators': 200}

In [25]:
scores = cross_val_score(grid, X_test, y_test, cv = 5, scoring = 'f1', n_jobs = -1)

In [26]:
print(scores, 'average',scores.mean())

[0.90909091 0.66666667 0.83333333 0.90909091 0.75      ] average 0.8136363636363637


In [27]:
from sklearn.metrics import classification_report,confusion_matrix

In [28]:
predictions = grid.predict(X_test)

In [29]:
print(confusion_matrix(y_test,predictions))

[[11  6]
 [ 0 26]]


In [30]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.65      0.79        17
           1       0.81      1.00      0.90        26

    accuracy                           0.86        43
   macro avg       0.91      0.82      0.84        43
weighted avg       0.89      0.86      0.85        43

